# Profiling Python's built-in `sorted()` and `list.sort()` methods

In [19]:
import time
import numpy as np
from typing import cast

rng = np.random.default_rng()
TEST_RUNS = 10
sizes = [10, 50, 100, 1000, 2500, 7500, 10000, 12500, 17500, 20000]

## Testing `sorted`

In [20]:
# Test on random data
for i in range(TEST_RUNS):
        n = sizes[i]
        arr = rng.integers(low=0, high=100, size=n).tolist()

        start = time.perf_counter()
        sorted(arr)
        end = time.perf_counter()

        print(f"n = {n:<8,} time = {(end - start) * 1_000:.3f} ms")

n = 10       time = 0.003 ms
n = 50       time = 0.003 ms
n = 100      time = 0.006 ms
n = 1,000    time = 0.074 ms
n = 2,500    time = 0.172 ms
n = 7,500    time = 0.526 ms
n = 10,000   time = 0.689 ms
n = 12,500   time = 0.866 ms
n = 17,500   time = 1.142 ms
n = 20,000   time = 1.306 ms


In [21]:
# Test on sorted data
for i in range(TEST_RUNS):
        n = sizes[i]
        arr = list(range(n))

        start = time.perf_counter()
        sorted(arr)
        end = time.perf_counter()

        print(f"n = {n:<8,} time = {(end - start) * 1_000:.3f} ms")

n = 10       time = 0.001 ms
n = 50       time = 0.001 ms
n = 100      time = 0.001 ms
n = 1,000    time = 0.004 ms
n = 2,500    time = 0.012 ms
n = 7,500    time = 0.034 ms
n = 10,000   time = 0.036 ms
n = 12,500   time = 0.057 ms
n = 17,500   time = 0.066 ms
n = 20,000   time = 0.073 ms



### Results

To figure out the expected runtime of `sorted`, we can look at ratios for common average-case runtimes:
* $\Theta(n)$
* $\Theta(\lg n)$
* $\Theta(n \lg n)$
* $\Theta(n^2)$

The results are as follows:

#### Random Data

| $n$    | $T(n)$    | $\dfrac{T(n)}{n}$ | $\dfrac{T(n)}{\lg n}$ | $\dfrac{T(n)}{n \lg n}$ | $\dfrac{T(n)}{n^2}$ |
| ------ | --------- | ----------------- | --------------------- | ----------------------- | ------------------- |
| 10     | 0.005     | 0.000500          | 0.001505              | 0.000150                | 0.000050            |
| 50     | 0.006     | 0.000120          | 0.001063              | 0.000021                | 0.0000024           |
| 100    | 0.013     | 0.000130          | 0.001958              | 0.000020                | 0.00000013          |
| 1,000  | 0.183     | 0.000183          | 0.01836               | 0.0000184               | 0.000000183         |
| 2,500  | 0.247     | 0.000099          | 0.02188               | 0.00000875              | 0.0000000395        |
| 7,500  | 1.142     | 0.000152          | 0.08874               | 0.0000118               | 0.0000000152        |
| 10,000 | 1.763     | 0.000176          | 0.1326                | 0.0000133               | 0.0000000176        |
| 12,500 | 1.931     | 0.000154          | 0.1418                | 0.0000113               | 0.0000000123        |
| 17,500 | 1.985     | 0.000113          | 0.1408                | 0.00000804              | 0.00000000649       |
| 20,000 | 2.391     | 0.000120          | 0.1674                | 0.00000593              | 0.00000000598       |

Looking at the ratios for $\Theta(n)$, we don't see the values stabilize at all. Similarly with $\Theta(n \lg n)$,
the ratios appear to increase exponentially as $n$ increases. Looking at $\Theta(n^2)$, the ratios appear to decrease
as $n$ increases. The ratios for $\Theta(n \lg n)$ are not perfect, but we can see that they tend to stabilize somewhere
between $1.0 \times 10^{-5}$ and $1.0 \times 10^{-6}$. This leads us to conclude that `sorted` runs in $\Theta(n \lg n)$ time.

#### Sorted Data

| $n$    | $T(n)$ (ms) | $\dfrac{T(n)}{n}$ | $\dfrac{T(n)}{\lg n}$ | $\dfrac{T(n)}{n \lg n}$ | $\dfrac{T(n)}{n^2}$ |
| ------ | ----------- | ----------------- | --------------------- | ----------------------- | ------------------- |
| 10     | 0.004       | 0.000400          | 0.001204              | 0.0001204               | 0.000040            |
| 50     | 0.003       | 0.000060          | 0.000091              | 0.00000182              | 0.0000012           |
| 100    | 0.002       | 0.000020          | 0.000301              | 0.00000301              | 0.0000002           |
| 1,000  | 0.013       | 0.000013          | 0.00131               | 0.00000131              | 0.000000013         |
| 2,500  | 0.028       | 0.0000112         | 0.00246               | 0.000000984             | 0.00000000448       |
| 7,500  | 0.122       | 0.0000163         | 0.00948               | 0.00000126              | 0.00000000216       |
| 10,000 | 0.084       | 0.0000084         | 0.00622               | 0.000000622             | 0.00000000084       |
| 12,500 | 0.102       | 0.00000816        | 0.00874               | 0.000000698             | 0.00000000065       |
| 17,500 | 0.159       | 0.00000909        | 0.01423               | 0.000000813             | 0.000000000519      |
| 20,000 | 0.136       | 0.0000068         | 0.00952               | 0.000000476             | 0.00000000034       |

When working with sorted data, the results become more interesting. For small values of $n$ ($\le 1000$), we can see that the runtimes
tend to stabilize around $1.0 \times 10^{-5}$. However, as $n$ gets much larger, we see a rapid decrease in runtime speed. Similar to
the run with random data, the ratios for $\Theta(n \lg n)$ appear to be the most stable for increasing values of $n$, settling around
$1.0 \times 10^{-7}$ in this case. This leads us to assume that `sorted` can run in $\Omega(n)$ time best-case, but for most large
problem sizes, the $\Theta(n \lg n)$ average-case runtime will dominate. 

## Testing `list.sort()`

In [ ]:
# Test on random data
for i in range(TEST_RUNS):
        n = sizes[i]
        arr = cast(list, rng.integers(low=0, high=100, size=n).tolist())

        start = time.perf_counter()
        arr.sort()
        end = time.perf_counter()

        print(f"n = {n:<8,} time = {(end - start) * 1_000:.3f} ms")

n = 10       time = 0.003 ms
n = 50       time = 0.004 ms
n = 100      time = 0.006 ms
n = 1,000    time = 0.062 ms
n = 2,500    time = 0.159 ms
n = 7,500    time = 0.722 ms
n = 10,000   time = 0.708 ms
n = 12,500   time = 1.110 ms
n = 17,500   time = 1.118 ms
n = 20,000   time = 1.276 ms


In [23]:
# Test on sorted data
for i in range(TEST_RUNS):
        n = sizes[i]
        arr = list(range(n))

        start = time.perf_counter()
        arr.sort()
        end = time.perf_counter()

        print(f"n = {n:<8,} time = {(end - start) * 1_000:.3f} ms")

n = 10       time = 0.002 ms
n = 50       time = 0.002 ms
n = 100      time = 0.001 ms
n = 1,000    time = 0.005 ms
n = 2,500    time = 0.012 ms
n = 7,500    time = 0.035 ms
n = 10,000   time = 0.072 ms
n = 12,500   time = 0.104 ms
n = 17,500   time = 0.324 ms
n = 20,000   time = 0.081 ms



### Results

We will analyze `list.sort` similar to how we analyzed `sorted` - by looking at ratios for common average-case runtimes:
* $\Theta(n)$
* $\Theta(\lg n)$
* $\Theta(n \lg n)$
* $\Theta(n^2)$

The results are as follows:

#### Random Data

| $n$    | $T(n)$ | $\dfrac{T(n)}{n}$ | $\dfrac{T(n)}{\lg n}$ | $\dfrac{T(n)}{n \lg n}$ | $\dfrac{T(n)}{n^2}$ |
| ------ | ------ | ----------------- | --------------------- | ----------------------- | ------------------- |
| 10     | 0.002  | 0.000200          | 0.000603              | 0.0000603               | 0.000020            |
| 50     | 0.004  | 0.000080          | 0.000709              | 0.0000141               | 0.0000016           |
| 100    | 0.007  | 0.000070          | 0.001053              | 0.0000105               | 0.0000007           |
| 1,000  | 0.090  | 0.000090          | 0.00904               | 0.000000906             | 0.000000090         |
| 2,500  | 0.229  | 0.0000916         | 0.02028               | 0.00000811              | 0.0000000366        |
| 7,500  | 0.904  | 0.0001205         | 0.07027               | 0.00000937              | 0.0000000161        |
| 10,000 | 1.042  | 0.0001042         | 0.07842               | 0.00000785              | 0.0000000104        |
| 12,500 | 1.171  | 0.0000937         | 0.08603               | 0.00000688              | 0.00000000749       |
| 17,500 | 1.840  | 0.000105          | 0.1305                | 0.00000746              | 0.00000000600       |
| 20,000 | 2.367  | 0.000118          | 0.1657                | 0.00000821              | 0.00000000592       |

#### Sorted Data

| $n$    | $T(n)$      | $\dfrac{T(n)}{n}$ | $\dfrac{T(n)}{\lg n}$  | $\dfrac{T(n)}{n \lg n}$  | $\dfrac{T(n)}{n^2}$ |
| ------ | ----------- | ----------------- | ---------------------- | ------------------------ | ------------------- |
| 10     | 0.002       | 0.000200          | 0.000601               | 0.000067                 | 0.0000200           |
| 50     | 0.002       | 0.000040          | 0.000177               | 0.0000089                | 0.0000008           |
| 100    | 0.001       | 0.000010          | 0.000100               | 0.0000010                | 0.0000001           |
| 1,000  | 0.005       | 0.000005          | 0.000050               | 0.00000050               | 0.000000005         |
| 2,500  | 0.012       | 0.0000048         | 0.000109               | 0.00000070               | 0.0000000019        |
| 7,500  | 0.035       | 0.0000047         | 0.000273               | 0.00000063               | 0.00000000062       |
| 10,000 | 0.072       | 0.0000072         | 0.000721               | 0.00000072               | 0.00000000072       |
| 12,500 | 0.104       | 0.0000083         | 0.000963               | 0.00000077               | 0.00000000067       |
| 17,500 | 0.324       | 0.0000185         | 0.00237                | 0.00000122               | 0.00000000106       |
| 20,000 | 0.081       | 0.0000041         | 0.000672               | 0.00000029               | 0.00000000020       |

Similar to the results for `sorted`, the ratios for $\Theta(n \lg n)$ are the most stable (somewhere
around $1.0 \times 10^{-7}$). This leads us to conclude that `list.sort` also runs in $\Theta(n \lg n)$
time. In this scenario, we see less stability for small problem sizes around $\Omega(n)$ best-case time,
but the ratios are fairly close (around $1.0 \times 10^{-5}$).

## Final Analysis

Both `sorted` and `list.sort` in Python are designed to use the same sorting algorithm, known as _Timsort_.
It's a hybrid algorithm, mirroring the behavior of both merge sort and insertion sort. It achieves
the following algorithmic time complexities:
* Best-case: $\Omega(n)$
* Average-case: $\Theta(n \lg n)$
* Worst-case: $\mathrm{O}(n \lg n)$

which are consistent with our experimental results.

In best-case scenarios (i.e., when used with sorted or near-sorted data), Timsort can achieve a best-case
time complexity of $\Omega(n)$. However, because of the hybrid nature of the algorithm, the insertion sort-style
behavior that Timsort experiences for small problem sizes quickly goes away as $n$ gets larger. This is why even
on sorted data, we didn't clearly see evidence that the algorithm's runtime was always $\Omega(n)$. Once the marge
sort-style behavior of the algorithm takes over, additional levels of complexity are added, as we know the generic
merge logic takes $\Omega(\lg n)$ time. Thus, even on sorted data, it's not out of the ordinary for these methods
to still run in $\Theta(n \lg n)$ time.

The two methods are similar, but have a few minor differences. While `list.sort` will only work on
instances of the `list` class, `sorted` is generic, and will sort any `iterable` whose elements are comparable.
The `list.sort` method performs all operations _in-place_, so the list it acts upon will be modified. The `sorted`
method, on the other hand, returns a new sorted collection - taking up $\mathrm{O}(n)$ auxillary space in the process.

_Source: https://en.wikipedia.org/wiki/Timsort_
